In [192]:
import timeit
import re
from operator import add

startt = timeit.default_timer()

path = '/export/usuarios01/sblanco/Datos/repogit/spiff_spark/'
path_out = '/export/usuarios01/sblanco/Datos/repogit/spiff_spark/Output/salida.csv'

In [193]:
def countWords ( cad ):
    return ( [word for word in re.findall(r'\b[^\W\d_]+\b',cad)])


def getData ( cad ):
    
        autor = '-1'
        match_menciones = []
        match_etiquetas = []
        match_urls = []
        
        salida = []
        try:
            cad = cad.strip ()
            
            #El autor es la primera arroba:    
            match = re.search (r'@[\w\.-]+', cad) 
            autor =  match.group(0)
            
            
            #Dejamos la cadena sin el autor para seguir buscando en ella:
            inicio_autor = cad.find('@') + 1
            fin_autor = cad.find(' ',inicio_autor)
            cad = cad[fin_autor+1:]

            #Buscamos todas las menciones que aparezcan:
            match_menciones = re.findall(r'@[\w\.-]+', cad)
            
            #y las etiquetas:
            match_etiquetas = re.findall(r'#[\w\.-]+', cad)

            #y las urls:
            match_urls = re.findall(r'(https?://[^\s]+)',cad)
            
            
            
            
        except Exception as e:
                match_menciones = [str(e)]
                match_urls = [cad]
        
        
        salida = [ match_menciones, match_etiquetas, match_urls ]
            
    
        #salida[0]: autor
        #salida[1][0]: lista de menciones, salida[1][0][0] primera mencion
        #salida[1][1]: etiquetas
        return autor, salida
    
def toCSV (rd_tweets, f):
    
    datasep = ';'
    listsep = ','
    
    myF = open (f, 'w')
    
    for tw in rd_tweets.collect():
        line = tw[0] + datasep
        
        for menc in tw[1][0]:
            line = line + menc + listsep
            
        line = line + datasep
        
        for etiqu in tw[1][1]:
            line = line + etiqu + listsep
        
        myF.write (line + '\n')
        
    myF.close()

    

In [194]:
#tweets = sc.textFile('file:///export/usuarios01/sblanco/Datos/repogit/spiff_spark/Data/03_10_2016/red.txt')
file_path = 'file://'+ path + '/Data/03_10_2016/todo.txt'
tweets = sc.textFile(file_path)

alldata = tweets.map ( getData )

#Errores procesados, o buscar un autor concreto:
author = '-1'
print 'tweets fallidos: %s' % (alldata.filter (lambda x : x[0]==author).count())


#Autor con mas entradas:
count = 10
moreentr = alldata.map ( lambda x: [x[0],1]).reduceByKey ( add ).takeOrdered( count ,key=lambda x: -x[1])
print 'Los %s autores con mas entradas son:%s' % (str(count), str(moreentr))

toCSV ( alldata, path_out)

tweets fallidos: 7879
Los 10 autores con mas entradas son:[('-1', 7879), (u'@TunisiaTrends', 1815), (u'@SBZ_news', 1638), (u'@nawaat', 1524), (u'@Dima_Khatib', 1482), (u'@weddady', 1466), (u'@LiveWordCanada', 1234), (u'@halmustafa', 1149), (u'@TunObs', 1114), (u'@WikiActions', 958)]


In [195]:
print 'Total time:%s' % (timeit.default_timer() - startt)

Total time:4.75341296196
